### 데이터 불러오기
메모리같은 제한된 자원으로 인해 모든 데이터를 메모리에 올려서 학습하기는 힘들기 때문에 일반적으로 배치 형태의 묶음으로 데이터를 나누어 모델 학습에 사용한다.

In [11]:
import torch # 파이토치 기본 라이브러리 
import torchvision # 이미지 관련 된 파이토치 라이브러리
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import numpy as np # 넘파이 기본 라이브러리
import matplotlib.pyplot as plt

In [12]:
# tr.Compose 내에 원하는 전처리를 차례대로 넣어주면 된다.

transf = tr.Compose([tr.Resize(16),tr.ToTensor()]) # 16x16으로 이미지 크기 변환 후 텐서 타입으로 변환한다.

Transforms on PIL Image  
Pad, Grayscale, RandomCrop, Normalize ..  
Transforms on torch.*Tensor - tensor image  
torchvision.transforms.ToPILImage(mode=None)...  

torchvision.datasets에서 제공하는 CIFAR10 데이터를 불러온다.  
root에는 다운로드 받을 경로를 입력한다.  
train=Ture이면 학습 데이터를 불러오고 train=False이면 테스트 데이터를 불러온다.  
미리 선언한 전처리를 사용하기 위해 transform=transf을 입력한다.  

In [13]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

Files already downloaded and verified
Files already downloaded and verified


일반적으로 데이터셋은 이미지와 라벨이 동시에 들어있는 튜플(tuple) 형태다. (이미지, 라벨)  
trainset[0]은 학습 데이터의 첫 번째 데이터로 이미지 한 장과 라벨 숫자 하나가 저장되어 있다.  
즉, trainset[0][0]은 이미지이며 trainset[0][1]은 라벨이다.  

In [14]:
print(trainset[0][0].size()) 

torch.Size([3, 16, 16])


현재 이미지 사이즈는 3x16x16이다. 여기서 3은 채널 수를 말하고 16x16은 이미지의 너비와 높이를 의미한다.  
일반적인 컬러 사진은 RGB 이미지이기 때문에 채널이 3개 이고 (너비)x(높이)x(채널 수)로 크기가 표현된다.  
하지만 파이토치에서는 이미지 한 장이 (채널 수)x(너비)x(높이)으로 표현되니 유의하도록 한다.

DataLoader는 데이터를 미니 배치 형태로 만들어 준다.  
따라서 배치 사이즈 및 셔플 여부 등을 선택할 수 있다.  

In [15]:
trainloader = DataLoader(trainset, batch_size=50, shuffle=True)
testloader = DataLoader(testset, batch_size=50, shuffle=False)

len(trainloader)

1000

CIFAR10의 학습 이미지는 50,000장이고 배치 사이즈가 50장이므로 1,000은 배치의 개수가 된다.  
즉 trainloader가 잘 만들어졌다는 것을 단편적으로 알 수 있다.  
iter, next를 이용해 일부 데이터를 확인할 수 있다.

In [16]:
images, labels = iter(trainloader).next()
print(images.size())

torch.Size([50, 3, 16, 16])


일반적으로 학습 데이터는 4차원 형태로 모델에서 사용된다.  
(배치 크기)x(채널 수)x(너비)x(높이)

In [ ]:
"""
oneshot = images[1].permute(1,2,0).numpy()
plt.figure(figsize=(2,2))
plt.imshow(oneshot)
plt.axis("off")
plt.show()
#컴퓨터에 따라서 이미지 호출하면 커널이 다운될 수 있음
"""

### 이미지를 같은 클래스(라벨)별로 폴더를 정리한 경우
데이터가 같은 클래스 별로 미리 폴더를 정리 된 경우, ImageFolder의 1줄 선언으로 개인 데이터를 사용할 수 있다.  
별도의 라벨링이 필요 없으며 폴더 별로 자동으로 라벨링을 한다.  
예를 들어 class 폴더에 tiger, lion 폴더(./class/tiger와 ./class/lion)를 미리 만든다.  
다음으로 ImageFolder에 상위 폴더 ./class를 입력하면 이미지와 라벨이 정리 되어 데이터를 불러온다.  

In [ ]:
"""
transf = tr.Compose([tr.Resize(128),tr.ToTensor()]) # 128x128 이미지 크기 변환 후 텐서로 만든다.
trainset = torchvision.datasets.ImageFolder(root='./class', transform=transf) # 커스텀 데이터 불러온다.
trainloader = DataLoader(trainset, batch_size=2, shuffle=False) # 데이터를 미니 배치 형태로 만들어 준다.

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(images.size(),labels)
"""

### 정리되지 않은 커스텀 데이터 불러오기
ImageFolder를 이용하면 매우 간단하게 이미지 데이터를 사용할 수 있지만 여러가지 이유로 사용이 불가능한 경우가 있다.  
1. 라벨 별로 폴더 정리가 되어 있으면 좋겠지만 그렇지 않은 경우가 많다.
2. 정리를 하고 싶지만 다른 작업들과 공유된 데이터인 경우 폴더를 함부로 정리할 수 없다.
3. 이미지 데이터라도 이미지가 아닌 텍스트, 리스트 ,배열 등의 다른 형태로 저장되어 있는 경우도 있다.

아래는 가장 기본적인 형태이다

from torch.utils.data import Dataset  

class MyDataset(Dataset):  <- Dataset을 상속받아 DataLoader에서 배치 단위로 불러 올 수 있게 해준다.
    
    def __init__(self):  <- 데이터 세팅에 필요한 것들을 정의
    
    def __getitem__(self, index):  <- Dataloader를 통해 샘플이 요청되면 __getitem__(self,index)은 인덱스에 해당하는 샘플을 찾아 준다
    
    def __len__(self):  <- 은 크기를 반환한다.

이 양식을 통으로 가지고 다니자!!

### 커스텀 데이터 세트 예시

In [17]:
# 32x32 컬러 이미지와 라벨이 각각 100장이 있다고 가정하다.

train_images = np.random.randint(256,size=(100,32,32,3))/255 # (이미지 수)x(너비)x(높이)x(채널 수)
train_labels = np.random.randint(2,size=(100,1)) # 라벨 수

class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data) # 이미지 데이터를 FloatTensor로 변형
        self.x_data = self.x_data.permute(0,3,1,2) # (이미지 수)x(너비)x(높이)x(채널 수) -> (배치 크기)x(채널 수)x(너비)x(높이)
        self.y_data = torch.LongTensor(y_data) # 라벨 데이터를 LongTensor로 변형
        self.len = self.y_data.shape[0] # 클래스 내의 들어 온 데이터 개수 

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index] # 뽑아 낼 데이터를 적어준다.

    def __len__(self):
        return self.len # 클래스 내의 들어 온 데이터 개수 

In [18]:
train_data = TensorData(train_images,train_labels) # 텐서 데이터 불러오기 
train_loader = DataLoader(train_data, batch_size=10, shuffle=True) # 미니 배치 형태로 데이터 갖추기

### 커스텀 데이터와 커스텀 전처리 사용하기
파이토치는 전처리 함수들을 제공하여 매우 편리하게 사용할 수 있다. 하지만 이미지의 경우 PIL-image 타입이거나 Tensor타입일 때 사용이 가능하다. 또한 제공하지 않는 기능에 대해서는 직접 구현이 필요하다. 이번 예시에서는 두개의 클래스를 직접 정의하고 사용한다  
먼저 텐서변환 전처리 클래스를 정의한다.

In [19]:
import torch
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import numpy as np # 넘파이 기본 라이브러리
import matplotlib.pyplot as plt

In [20]:
# 1. 텐서 변환
class ToTensor:
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs) # 텐서로 변환
        inputs = inputs.permute(2,0,1) # 크기 변환
        return inputs, torch.LongTensor(labels) # 텐서로 변환

CutOut은 이미지 내부에 무작위로 사각형 영역을 선택하여 0으로 만드는 데이터 증식 방법이다.

In [21]:
# 2. CutOut    
class CutOut:
    
    def __init__(self, ratio=.5):
        self.ratio = int(1/ratio)
        # __init__함수를 사용하여 ratio를 받는다. 기본 ratio는 0.5로 세팅하면 불러온 이미지에 대해 50% 확률로 CutOut을 발현
           
    def __call__(self, sample): # 샘플을 받는다.
        inputs, labels = sample
        active = int(np.random.randint(0, self.ratio, 1))
        
        if active == 0:
            _, w, h = inputs.size()
            min_len = min(w, h)
            box_size = int(min_len//4)
            idx = int(np.random.randint(0, min_len-box_size, 1))
            inputs[:,idx:idx+box_size,idx:idx+box_size] = 0
        
        return inputs, labels

In [22]:
# 위에서 사용한 양식을 그대로 사용하되 전처리 작업을 할 수 있도록 transform을 추가한다. 
class MyDataset(Dataset):
    
    def __init__(self, x_data, y_data, transform=None):
        
        self.x_data = x_data # 넘파이 배열이 들어온다.
        self.y_data = y_data # 넘파이 배열이 들어온다.
        self.transform = transform
        self.len = len(y_data)
        self.tensor = ToTensor()
    
    def __getitem__(self, index):
        sample = self.x_data[index], self.y_data[index]
        
        if self.transform:
            sample = self.transform(sample) #self.transform이 None이 아니라면 전처리를 작업한다.
        else:
            sample = self.tensor(sample)
        
        return sample 
    
    def __len__(self):
        return self.len       

In [23]:
#trans = tr.Compose([ToTensor(),LinearTensor(2,5)]) # 텐서 변환 후 선형식 2x+5 연산
trans = tr.Compose([ToTensor(),CutOut()]) 
dataset1 = MyDataset(train_images,train_labels, transform=trans)
train_loader1 = DataLoader(dataset1, batch_size=10, shuffle=True)

# ToTensor()와 tr.ToTensor()의 차이
# 앞 서 사용한 tr.ToTensor()는 import torchvision.transforms as tr를 이용한 파이토치 메소드를 이용한 것이고
# ToTensor()는 위에서 정의 된 메소드를 사용한 것이다.

In [24]:
images1, labels1 = iter(train_loader1).next()
print(images1.size()) # 배치 및 이미지 크기 확인

torch.Size([10, 3, 32, 32])


In [25]:
import torchvision
def imshow(img):
    plt.figure(figsize=(10,100))
    plt.imshow(img.permute(1,2,0).numpy())
    plt.show()

In [26]:
#imshow(torchvision.utils.make_grid(images1,nrow=10))

### 커스텀 데이터와 파이토치 제공 전처리 사용하기
텐서 변환과 같은 전처리는 파이토치에서 제공하는 전처리를 사용하면 편리하다. 하지만 앞서 언급했듯이 ㅏㅍ이토치에서 제공되는 많은 전처리는 PILImage 타입일 경우 사용할 수 있다. 따라서 기능은 있는데 데이터 타입이 다른 경우는 PILImage 타입으로 변환하여 제공된 전처리를 사용할 수 있다.

In [27]:
# torchvision.transforms에서 제공하는 전처리 기술을 사용한다.
# torchvision.transforms은 입력 이미지가 일반적으로 PILImage 타입이나 텐서일 경우에 동작한다.
# 현재 데이터는 넘파이 배열이다. 따라서 텐서 변환 후 tr.ToPILImage()을 이용하여 PILImage 타입으로 만들어 준다.
# __call__을 이용한 기본 구조는 동일하다.

class MyTransform:
    
    def __call__(self, sample):
        inputs, labels = sample
        inputs = torch.FloatTensor(inputs)
        inputs = inputs.permute(2,0,1)
        labels = torch.FloatTensor(labels)

        transf = tr.Compose([tr.ToPILImage(), tr.Resize(128), tr.ToTensor()])
        final_output = transf(inputs)      
        
        return final_output, labels  

In [28]:
dataset2 = MyDataset(train_images,train_labels, transform=MyTransform())
train_loader2 = DataLoader(dataset2, batch_size=10, shuffle=True)

### 커스텀 데이터와 파이토치 제공 전처리 사용하기

In [29]:
class CutOut:
    
    def __init__(self, ratio=.5):
        self.ratio = int(1/ratio)
           
    def __call__(self, inputs):

        active = int(np.random.randint(0, self.ratio, 1))
        
        if active == 0:
            _, w, h = inputs.size()
            min_len = min(w, h)
            box_size = int(min_len//4)
            idx = int(np.random.randint(0, min_len-box_size, 1))
            inputs[:,idx:idx+box_size,idx:idx+box_size] = 0
        
        return inputs

In [ ]:
trainloader = DataLoader(trainset, batch_size=10, shuffle=True)
images, labels = iter(trainloader).next()
imshow(torchvision.utils.make_grid(images,nrow=10))
print(images.size()) # 배치 및 이미지 크기 확인